<a href="https://colab.research.google.com/gist/louismullie/ef8e230f750e0eb3c6b36c64263f99e0/base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  
!mkdir -p ~/.kaggle                                          
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log 
!cp kaggle.json ~/.kaggle/kaggle.json > log              
!chmod 600 ~/.kaggle/kaggle.json                            
!kaggle config set -n competition -v novozymes-enzyme-stability-prediction   
!kaggle competitions download -c novozymes-enzyme-stability-prediction         
!unzip -o *.zip >> log                           
!kaggle competitions leaderboard --show                

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: novozymes-enzyme-stability-prediction
novozymes-enzyme-stability-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)
Using competition: novozymes-enzyme-stability-prediction
 teamId  teamName                                        submissionDate       score   
-------  ----------------------------------------------  -------------------  ------  
9186580  Chris Deotte                                    2022-10-04 01:56:00  0.504   
9184261  Muhammad Ahmed                                  2022-10-03 10:32:04  0.502   
9200132  enzymedesign                                    2022-10-03 04:09:11  0.500   
9234521  Proletheus                                      2022-10-03 13:53:48  0.496   
9181891  ‎‎‎‎‎‎‎‎‎‎‎‎‎三千鸦杀‎                              2022-10-04 00:57:39  0.479   
9224814  deltaTM                                         2022-10-03 12:25:51  0.473  

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', low_memory=False); df

,seq_id,protein_sequence,pH,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.7


In [4]:
characters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
char_to_num = {}
for x in range(len(characters)):
    char_to_num[characters[x]] = x

def frequency_by_letter(protein_sequence):
  frequencies = np.zeros((26))
  for letter in protein_sequence:
    char_num = char_to_num[letter]
    frequencies[char_num] = frequencies[char_num] + 1
  return frequencies

letter_frequencies = df.protein_sequence.apply(frequency_by_letter)

In [5]:
df.insert(3, 'letter_frequencies', letter_frequencies, True)
df

,seq_id,protein_sequence,pH,letter_frequencies,data_source,tm
0,0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,"[45.0, 0.0, 1.0, 13.0, 30.0, 13.0, 38.0, 3.0, ...",doi.org/10.1038/s41592-020-0801-4,75.7
1,1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,"[28.0, 0.0, 0.0, 10.0, 52.0, 6.0, 18.0, 4.0, 1...",doi.org/10.1038/s41592-020-0801-4,50.5
2,2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,"[50.0, 0.0, 9.0, 27.0, 32.0, 21.0, 65.0, 11.0,...",doi.org/10.1038/s41592-020-0801-4,40.5
3,3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,"[20.0, 0.0, 5.0, 19.0, 29.0, 12.0, 16.0, 7.0, ...",doi.org/10.1038/s41592-020-0801-4,47.2
4,4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,"[86.0, 0.0, 14.0, 78.0, 78.0, 32.0, 84.0, 40.0...",doi.org/10.1038/s41592-020-0801-4,49.5
...,...,...,...,...,...,...
31385,31385,YYMYSGGGSALAAGGGGAGRKGDWNDIDSIKKKDLHHSRGDEKAQG...,7.0,"[33.0, 0.0, 12.0, 38.0, 31.0, 18.0, 51.0, 15.0...",doi.org/10.1038/s41592-020-0801-4,51.8
31386,31386,YYNDQHRLSSYSVETAMFLSWERAIVKPGAMFKKAVIGFNCNVDLI...,7.0,"[37.0, 0.0, 5.0, 21.0, 29.0, 22.0, 27.0, 22.0,...",doi.org/10.1038/s41592-020-0801-4,37.2
31387,31387,YYQRTLGAELLYKISFGEMPKSAQDSAENCPSGMQFPDTAIAHANV...,7.0,"[13.0, 0.0, 1.0, 7.0, 7.0, 7.0, 11.0, 2.0, 6.0...",doi.org/10.1038/s41592-020-0801-4,64.6
31388,31388,YYSFSDNITTVFLSRQAIDDDHSLSLGTISDVVESENGVVAADDAR...,7.0,"[47.0, 0.0, 5.0, 34.0, 36.0, 23.0, 52.0, 11.0,...",doi.org/10.1038/s41592-020-0801-4,50.7


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

model = RandomForestRegressor(max_depth=500, random_state=0)

X = np.stack(df.letter_frequencies.values, axis=0)
y = np.asarray(df.tm.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

print(X_train.shape)
print(y_train.shape)

model.fit(X_train, y_train)

(15695, 26)
(15695,)


RandomForestRegressor(max_depth=500, random_state=0)

In [7]:
from scipy.stats import spearmanr

y_pred = model.predict(X_test)
y_true = y_test

correlation = spearmanr(y_true,y_pred)[0]
print('Spearman R on test set: %f' % correlation)

Spearman R on test set: 0.535148


In [8]:
df_test = pd.read_csv('test.csv', low_memory=False); df_test
letter_frequencies = df_test.protein_sequence.apply(frequency_by_letter)
df_test.insert(3, 'letter_frequencies', letter_frequencies, True)
predictions = model.predict(np.stack(df_test.letter_frequencies,axis=0))


ToCSV = lambda df_tmp, fname: df_tmp.round(2).to_csv(f'{fname}.csv', index_label='seq_id')
pY = pd.DataFrame(predictions, index=range(31390,len(predictions)+31390), columns=['tm'])
ToCSV(pY, '/content/drive/MyDrive/Kaggle/Novozymes_Submission')